<a href="https://colab.research.google.com/github/ideablast/NLPer_transformer_doc2vec_chatbot/blob/toram/bilstm_classification_chatbot_final_version.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install konlpy
!pip install git+https://github.com/haven-jeon/PyKoSpacing.git
!pip install git+https://github.com/ssut/py-hanspell.git

     |████████████████████████████████| 19.4MB 1.3MB/s 
     |████████████████████████████████| 460kB 37.0MB/s 
     |████████████████████████████████| 92kB 8.0MB/s 
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
  Cloning https://github.com/haven-jeon/PyKoSpacing.git to /tmp/pip-req-build-g1vwat1l
  Running command git clone -q https://github.com/haven-jeon/PyKoSpacing.git /tmp/pip-req-build-g1vwat1l
  Created wheel for pykospacing: filename=pykospacing-0.3-cp36-none-any.whl size=2255638 sha256=7f366a49bfc4adcb5fd3b180f72f2de56098e50a5e66773b4c06d67c0e7243e3
  Stored in directory: /tmp/pip-ephem-wheel-cache-_we90sia/wheels/4d/45/58/e26cb2b7f6a063d234158c6fd1e5700f6e15b99d67154340ba
Successfully built pykospacing


  Cloning https://github.com/ssut/py-hanspell.git to /tmp/pip-req-build-57r6wp3_
  Running command git clone -q https://github.com/ssut/py-hanspell.git /tmp/pip-req-build-57r6wp3_
  Created wheel for py-hanspell: filename=py_hanspell-1.1-cp36-none-any.whl size=4854 sha256=437364cf00a26056a1fa9a6825792ac04f8fa0775f7514ed83e8172e0a20c223
  Stored in directory: /tmp/pip-ephem-wheel-cache-30qa7ffj/wheels/0a/25/d1/e5e96476dbb1c318cc26c992dd493394fe42b0c204b3e65588
Successfully built py-hanspell


## Import

In [3]:
from keras import models
from keras import layers
from keras import optimizers, losses, metrics
from keras import preprocessing

import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import re

from konlpy.tag import Okt
from hanspell import spell_checker
from pykospacing import spacing

import matplotlib.pyplot as plt

## Hyperparameter

In [4]:
# 태그 단어
PAD = "<PADDING>"   # 패딩
STA = "<START>"     # 시작
END = "<END>"       # 끝
OOV = "<OOV>"       # 없는 단어(Out of Vocabulary)

# 태그 인덱스
PAD_INDEX = 0
STA_INDEX = 1
END_INDEX = 2
OOV_INDEX = 3

# 데이터 타입
ENCODER_INPUT  = 0
DECODER_INPUT  = 1
DECODER_TARGET = 2

# Hyper-parameters
NUM_LAYERS = 2
D_MODEL = 256 ##word embedding dim
NUM_HEADS = 8 ## D_Model % NUM_HEADS == 0이 되야하므로...
UNITS = 512 ## FFNN 유닛수
DROPOUT = 0.1
EPOCHS = 50 ## Transformer 문장생성기 에폭
# for Transformer data pipelining
BATCH_SIZE = 64
BUFFER_SIZE = 1000

VOCAB_SIZE = 0 # 후에 len(words) 로 바뀜.

# 한 문장에서 단어 시퀀스의 최대 개수
max_sequences = 30

# 정규 표현식 필터
RE_FILTER = re.compile("[\"':;~()]")

## Data Load & Preprocessing

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
wear_data = pd.read_csv("/content/drive/My Drive/wear.csv")
print(wear_data.shape)
customer = wear_data[wear_data.SPEAKER == "고객"].SENTENCE
store = wear_data[wear_data.SPEAKER == "점원"].SENTENCE
print(customer.shape, store.shape) # 질문의 개수와 답의 개수가 일치하지 않는다.

(15826, 20)
(8381,) (7445,)


In [8]:
prev = "고객"
store_arr = []
customer_arr = []
store_stc = ""
customer_stc = ""
customer_C = "" #고객의 마지막 문장의 CATEGORY
customer_M = "" #고객의 마지막 문장의 의도

c_m = []
for i in range(wear_data.shape[0]):
    customer_C = wear_data.iloc[i].CATEGORY
    customer_M = wear_data.iloc[i].MAIN

    if (prev == wear_data.iloc[i].SPEAKER):
        if prev == "점원":
             store_stc += (" "+wear_data.iloc[i].SENTENCE)
        else : 
             customer_stc += (" "+wear_data.iloc[i].SENTENCE)
            
    elif prev == "점원": # 점원 -> 고객
        store_arr.append(store_stc)
        customer_stc = wear_data.iloc[i].SENTENCE
        prev = "고객"
    else : # 고객 -> 점원
        customer_arr.append(customer_stc)
        c_m.append([customer_C,customer_M])
        store_stc = wear_data.iloc[i].SENTENCE
        prev = "점원"

print(c_m)

[['신발', '종류별신발제품문의요청'], ['신발', '착화감'], ['신발', '착화감'], ['신발', '굽높이문의'], ['신발', '재입고문의'], ['신발', '소재문의'], ['신발', '제품가격문의'], ['신발', '사이즈재고문의'], ['신발', '재입고문의'], ['가방', '종류별가방제품문의요청'], ['가방', '제품가격문의'], ['가방', '소재를제시한제품문의'], ['가방', '제품가격문의'], ['가방', '가방소재문의'], ['가방', '제품가격문의'], ['가방', '제품구성문의'], ['가방', '공휴일영업문의'], ['가방', '공휴일영업문의'], ['가방', '제품가격문의'], ['액세서리', '제품소재문의'], ['액세서리', '세트제품개별구매문의'], ['액세서리', '제품소재문의'], ['액세서리', '제품가격문의'], ['액세서리', '종류별액세서리제품문의요청'], ['액세서리', '계절상품문의'], ['액세서리', '제품문의'], ['액세서리', '제품가격문의'], ['액세서리', '매장에서착용문의'], ['액세서리', '신상품문의'], ['액세서리', '제품의세척방법문의'], ['액세서리', '제품문의'], ['액세서리', '마감시간문의'], ['액세서리', '공휴일영업문의'], ['액세서리', '종류별액세서리제품문의요청'], ['액세서리', '제품의세척방법문의'], ['액세서리', '원단문의'], ['액세서리', '제품가격문의'], ['액세서리', '종류별액세서리제품문의요청'], ['액세서리', '제품가격문의'], ['액세서리', '디자인문의'], ['액세서리', '상품권사용문의'], ['액세서리', '종류별액세서리제품문의요청'], ['액세서리', '소재를제시한제품문의'], ['액세서리', '제품문의'], ['액세서리', '제품가격문의'], ['액세서리', '제품가격문의'], ['액세서리', '제품문의'], ['액세서리', '제품사용방법문의'], ['액세서리', '제품가격문의'], ['액세서리', '종류별액세

In [9]:
faqs = []
for i in range(len(store_arr)):
    faqs_tmp =[]
    faqs_tmp.append(str(i+1))
    faqs_tmp.append(customer_arr[i])
    faqs_tmp.append(store_arr[i])
    faqs_tmp.extend(c_m[i])
    faqs.append(faqs_tmp)
print(faqs)

[['1', ' 신발은 여기 있는 게 다예요?', '네 성인이나 아동 다 있어요 발 사이즈 몇 신으세요?', '신발', '종류별신발제품문의요청'], ['2', '230이요', '편하게 신을 수 있는 거 찾으세요?', '신발', '착화감'], ['3', '네 봄이니까 편하게 신을 수 있는 거', '이런 건 어떠세요? 이런 거도 신발 무척 편하거든요', '신발', '착화감'], ['4', '굽 좀 높은 거 없나요?', '봄 상품은 아직 어른 제품이 많이 안나왔습니다', '신발', '굽높이문의'], ['5', '언제 들어와요?', '이번주 지나면 들어올 거예요', '신발', '재입고문의'], ['6', '이거는 가죽이에요?', '가죽 아니고 쎄무예요', '신발', '소재문의'], ['7', '가죽은 얼마예요?', '2만 9천 원입니다', '신발', '제품가격문의'], ['8', '털 달린 거 저거는 사이즈 있어요?', '230이 없어요  이거 한 번 신어보세요', '신발', '사이즈재고문의'], ['9', '좀 크네 또 안 들어와요?', '네 이건 다 끝났어요', '신발', '재입고문의'], ['10', '가방 매는 거 보고 있어요', '여기 있어요', '가방', '종류별가방제품문의요청'], ['11', '가격이 얼마예요?', '이 종류는 2만 원이고 이 종류는 3만 8천 원이에요', '가방', '제품가격문의'], ['12', '가죽으로 된 거는 없어요?', '가죽은 없고 레자만 있어요', '가방', '소재를제시한제품문의'], ['13', '레자는 얼마예요?', '5만 5천 원요', '가방', '제품가격문의'], ['14', '이거는 천이죠?', '네 맞아요', '가방', '가방소재문의'], ['15', '이건 얼마예요?', '그것도 5만 5천 원요', '가방', '제품가격문의'], ['16', '이거 끈은 따로 없어요?', '안에 있어요', '가방', '제품구성문의'], ['17', '내일은 문 열어요?', '휴무입니다', '가방', '공휴일영업문의'], ['18

In [10]:
category_arr = []
main_arr = []
for i in faqs:
  category_arr.append(i[3])
  main_arr.append(i[4])

category_set = set(category_arr)
main_set = set(main_arr)

category_to_index = {word: index for index, word in enumerate(category_set)}
main_to_index = {word: index for index, word in enumerate(main_set)} #질문의 의도만 가져오다보니 405개에서 387개로 줄어들음

index_to_category = {index: word for index, word in enumerate(category_set)}
index_to_main = {index: word for index, word in enumerate(main_set)}

In [11]:
for i,item in enumerate(faqs):
  faqs[i][3] = category_to_index[item[3]]
  faqs[i][4] = main_to_index[item[4]]

In [12]:
question = []
answer = []

for Q in customer_arr:
    question.append(Q.replace("[^\w]", " "))

for A in store_arr:
    answer.append(A.replace("[^\w]", " "))

len(question), len(answer)

(7301, 7301)

In [13]:
question[:10]

[' 신발은 여기 있는 게 다예요?',
 '230이요',
 '네 봄이니까 편하게 신을 수 있는 거',
 '굽 좀 높은 거 없나요?',
 '언제 들어와요?',
 '이거는 가죽이에요?',
 '가죽은 얼마예요?',
 '털 달린 거 저거는 사이즈 있어요?',
 '좀 크네 또 안 들어와요?',
 '가방 매는 거 보고 있어요']

In [14]:
answer[:10]

['네 성인이나 아동 다 있어요 발 사이즈 몇 신으세요?',
 '편하게 신을 수 있는 거 찾으세요?',
 '이런 건 어떠세요? 이런 거도 신발 무척 편하거든요',
 '봄 상품은 아직 어른 제품이 많이 안나왔습니다',
 '이번주 지나면 들어올 거예요',
 '가죽 아니고 쎄무예요',
 '2만 9천 원입니다',
 '230이 없어요  이거 한 번 신어보세요',
 '네 이건 다 끝났어요',
 '여기 있어요']

In [15]:
# 형태소분석 함수
def pos_tag(sentences):
    
    # KoNLPy 형태소분석기 설정
    tagger = Okt()
    
    # 문장 품사 변수 초기화
    sentences_pos = []
    
    # 모든 문장 반복
    for sentence in sentences:
        # [\"':;~()] 특수기호 제거
        sentence = re.sub(RE_FILTER, "", sentence)
        
        # 배열인 형태소분석의 출력을 띄어쓰기로 구분하여 붙임
        sentence = " ".join(tagger.morphs(sentence))
        sentences_pos.append(sentence)
        
    return sentences_pos

In [16]:
# 형태소분석 수행
question = pos_tag(question)
answer = pos_tag(answer)

# 형태소분석으로 변환된 챗봇 데이터 출력
for i in range(5):
    print('Q : ' + question[i])
    print('A : ' + answer[i])
    print()


Q : 신발 은 여기 있는 게 다예 요 ?
A : 네 성인 이나 아동 다 있어요 발 사이즈 몇 신으세요 ?

Q : 230 이요
A : 편하게 신 을 수 있는 거 찾으세요 ?

Q : 네 봄 이니까 편하게 신 을 수 있는 거
A : 이런 건 어떠세요 ? 이런 거도 신발 무척 편하거든요

Q : 굽 좀 높은 거 없나요 ?
A : 봄 상품 은 아직 어른 제품 이 많이 안 나왔습니다

Q : 언제 들어와요 ?
A : 이번 주 지나면 들어올 거 예요



In [17]:
# 질문과 대답 문장들을 하나로 합침
sentences = []
sentences.extend(question)
sentences.extend(answer)

words = [] # 단어사전

# 단어들의 배열 생성
for sentence in sentences:
    for word in sentence.split():
        words.append(word)

# 길이가 0인 단어는 삭제
words = [word for word in words if len(word) > 0]

# 중복된 단어 삭제
words = list(set(words))

# 제일 앞에 태그 단어 삽입
words[:0] = [PAD, STA, END, OOV]

In [18]:
VOCAB_SIZE = len(words)
print("손님과 점원의 말에서 사용된 총 단어의 수 :",len(words))

손님과 점원의 말에서 사용된 총 단어의 수 : 6409


In [19]:
words[:10]

['<PADDING>',
 '<START>',
 '<END>',
 '<OOV>',
 '가능해요',
 '보드라운',
 '열',
 '줄이는거',
 '폴리',
 '받고']

In [20]:
# 단어와 인덱스의 딕셔너리 생성
word_to_index = {word: index for index, word in enumerate(words)}
index_to_word = {index: word for index, word in enumerate(words)}

In [21]:
# 문장을 인덱스로 변환
def convert_text_to_index(sentences, vocabulary, type): 
    
    sentences_index = []
    
    # 모든 문장에 대해서 반복
    for sentence in sentences:
        sentence_index = []
        
        # 디코더 입력일 경우 맨 앞에 START 태그 추가
        if type == DECODER_INPUT:
            sentence_index.extend([vocabulary[STA]])
        
        # 문장의 단어들을 띄어쓰기로 분리
        for word in sentence.split():
            if vocabulary.get(word) is not None:
                # 사전에 있는 단어면 해당 인덱스를 추가
                sentence_index.extend([vocabulary[word]])
            else:
                # 사전에 없는 단어면 OOV 인덱스를 추가
                sentence_index.extend([vocabulary[OOV]])

        # 최대 길이 검사
        if type == DECODER_TARGET:
            # 디코더 목표일 경우 맨 뒤에 END 태그 추가
            if len(sentence_index) >= max_sequences:
                sentence_index = sentence_index[:max_sequences-1] + [vocabulary[END]]
            else:
                sentence_index += [vocabulary[END]]
        else:
            if len(sentence_index) > max_sequences:
                sentence_index = sentence_index[:max_sequences]
            
        # 최대 길이에 없는 공간은 패딩 인덱스로 채움
        sentence_index += (max_sequences - len(sentence_index)) * [vocabulary[PAD]]
        
        # 문장의 인덱스 배열을 추가
        sentences_index.append(sentence_index)

    return np.asarray(sentences_index)

In [22]:
# 인코더 입력 인덱스 변환
x_encoder = convert_text_to_index(question, word_to_index, ENCODER_INPUT)

# 첫 번째 인코더 입력 출력 (신발 은 여기 있는 게 다예 요)
print(question[0])
x_encoder[0]


신발 은 여기 있는 게 다예 요 ?


array([3173, 4944, 3793,   55, 6124, 2879, 6011, 2048,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0])

In [23]:
# 디코더 입력 인덱스 변환
x_decoder = convert_text_to_index(answer, word_to_index, DECODER_INPUT)

# 첫 번째 디코더 입력 출력 (<START> 신발 은 여기 있는 게 다예 요)
print(question[0])
x_decoder[0]


신발 은 여기 있는 게 다예 요 ?


array([   1, 3197, 2031, 1764, 5319, 1891, 4279, 5486, 3993, 3500, 3230,
       2048,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0])

In [24]:
# 디코더 목표 인덱스 변환
y_decoder = convert_text_to_index(answer, word_to_index, DECODER_TARGET)

# 첫 번째 디코더 입력 출력 (신발 은 여기 있는 게 다예 요 <END>)
print(question[0])
y_decoder[0]


신발 은 여기 있는 게 다예 요 ?


array([3197, 2031, 1764, 5319, 1891, 4279, 5486, 3993, 3500, 3230, 2048,
          2,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0])

## Transformer 설계
 - data pipelining
 - Encoder layer * 2 => Encoder
 - Decoder layer * 2 => Decoder
 - Encoder + Decoder => Transformer

In [78]:
# decoder inputs use the previous target as input
dataset = tf.data.Dataset.from_tensor_slices((
    {
        'inputs': x_encoder,
        'dec_inputs': x_decoder
    },
    {
        'outputs': y_decoder
    },
))

dataset = dataset.cache()
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE)
dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)

$$\Large{Attention(Q, K, V) = softmax_k(\frac{QK^T}{\sqrt{d_k}}) V} $$

In [79]:
## scaled dot product Attention
def scaled_dot_product_attention(query, key, value, mask):
  matmul_qk = tf.matmul(query, key, transpose_b=True) # QK^T

  depth = tf.cast(tf.shape(key)[-1], tf.float32)
  logits = matmul_qk / tf.math.sqrt(depth) #  QK^T / sqrt(d_k)

  if mask is not None:
    logits += (mask * -1e9) # zero padding token softmax 결과가 0이 나오도록
  
  attention_weights = tf.nn.softmax(logits, axis = -1) # softmax(QK^T / sqrt(d_k))

  output = tf.matmul(attention_weights, value) # softmax(QK^T / sqrt(d_k)) * V

  return output

In [80]:
## multi-head attention
## each head need (scaled_dot_product_attention)
class MultiHeadAttention(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads, name="multi_head_attention"):
    super(MultiHeadAttention, self).__init__(name=name)
    self.num_heads = num_heads
    self.d_model = d_model

    assert d_model % self.num_heads == 0 # 128,8

    self.depth = d_model // self.num_heads

    self.query_dense = tf.keras.layers.Dense(units=d_model)
    self.key_dense = tf.keras.layers.Dense(units=d_model)
    self.value_dense = tf.keras.layers.Dense(units=d_model)

    self.dense = tf.keras.layers.Dense(units=d_model)
  
  def get_config(self):
      config = super().get_config().copy()
      config.update({
          'num_heads' : self.num_heads,
          'd_model': self.d_model,
          'depth' : self.depth
      })
      return config


  def split_heads(self, inputs, batch_size):
    inputs = tf.reshape(inputs, shape=(batch_size,-1,self.num_heads, self.depth))
    return tf.transpose(inputs, perm=[0, 2, 1, 3]) ##????
  
  def call(self, inputs):
    query, key, value, mask = inputs['query'], inputs['key'], inputs['value'], inputs['mask']
    batch_size = tf.shape(query)[0]

    #linear
    query = self.query_dense(query)
    key = self.key_dense(key)
    value = self.value_dense(value)

    #split heads
    query = self.split_heads(query, batch_size)
    key = self.split_heads(key, batch_size)
    value = self.split_heads(value, batch_size)

    #scaled dot-product attention
    scaled_attention = scaled_dot_product_attention(query, key, value, mask)
    scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])

    #concatenation of heads
    concat_attention = tf.reshape(scaled_attention, (batch_size, -1, self.d_model))

    #final linear
    outputs = self.dense(concat_attention)

    return outputs

In [81]:
def create_padding_mask(x):
  mask = tf.cast(tf.math.equal(x, 0), tf.float32)
  # (batch_size, 1, 1, sequence length)
  return mask[:, tf.newaxis, tf.newaxis, :]

In [82]:
print(create_padding_mask(tf.constant([[1, 2, 0, 3, 0], [0, 0, 0, 4, 5]])))

tf.Tensor(
[[[[0. 0. 1. 0. 1.]]]


 [[[1. 1. 1. 0. 0.]]]], shape=(2, 1, 1, 5), dtype=float32)


In [83]:
# it handle mask future tokens in a sequence used decoder. and mask pad tokens
def create_look_ahead_mask(x):
  seq_len = tf.shape(x)[1]
  look_ahead_mask = 1 - tf.linalg.band_part(tf.ones((seq_len, seq_len)), -1, 0)
  padding_mask = create_padding_mask(x)
  return tf.maximum(look_ahead_mask, padding_mask)

In [84]:
print(create_look_ahead_mask(tf.constant([[1, 2, 0, 4, 5]])))

tf.Tensor(
[[[[0. 1. 1. 1. 1.]
   [0. 0. 1. 1. 1.]
   [0. 0. 1. 1. 1.]
   [0. 0. 1. 0. 1.]
   [0. 0. 1. 0. 0.]]]], shape=(1, 1, 5, 5), dtype=float32)


Positional encoding

since we don't use any rnn, cnn, positional encoding give model position information of words in sentence.

positional encoding vector is added to embedding vector

$$\Large{PE_{(pos, 2i)} = sin(pos / 10000^{2i / d_{model}})} $$
$$\Large{PE_{(pos, 2i+1)} = cos(pos / 10000^{2i / d_{model}})} $$

In [85]:
class PositionalEncoding(tf.keras.layers.Layer):
  def __init__(self, position, d_model):
    super(PositionalEncoding, self).__init__()
    self.pos_encoding = self.positional_encoding(position, d_model)
    self.position = position
    self.d_model = d_model

  def get_config(self):
        config = super().get_config().copy()
        config.update({
            'position': self.position,
            'd_model': self.d_model,
        })
        return config
  
  def get_angles(self, position, i, d_model):
    angles = 1 / tf.pow(10000, (2 * (i // 2)) / tf.cast(d_model, tf.float32))
    return position * angles #pos/10000^(2i/d_model)

  def positional_encoding(self, position, d_model):
    angle_rads = self.get_angles(
        position = tf.range(position, dtype=tf.float32)[:, tf.newaxis],
        i = tf.range(d_model, dtype=tf.float32)[tf.newaxis, :],
        d_model = d_model)
    sines = tf.math.sin(angle_rads[:, 0::2])
    cosines = tf.math.cos(angle_rads[:, 1::2])

    pos_encoding = tf.concat([sines, cosines], axis=-1)
    pos_encoding = pos_encoding[tf.newaxis, ...]
    return tf.cast(pos_encoding, tf.float32)
  
  def call(self, inputs):
    return inputs + self.pos_encoding[:, :tf.shape(inputs)[1], :]

### Encoder Layer
1. Multi-head attention (with padding mask)
2. 2 dense layers followed by dropout

also has residual connection followd by a layer normalization.

In [86]:
def encoder_layer(units, d_model, num_heads, dropout, name="encoder_layer"):
  inputs = tf.keras.Input(shape=(None, d_model), name="inputs")
  padding_mask = tf.keras.Input(shape=(1, 1, None), name="padding_mask")

  attention = MultiHeadAttention(
      d_model, num_heads, name="attention")({
          'query':inputs,
          'key':inputs,
          'value':inputs,
          'mask':padding_mask
      })
  attention = tf.keras.layers.Dropout(rate=dropout)(attention)
  attention = tf.keras.layers.LayerNormalization(epsilon=1e-6)(inputs + attention)

  outputs = tf.keras.layers.Dense(units=units, activation='relu')(attention)
  outputs = tf.keras.layers.Dense(units=d_model)(outputs)
  outputs = tf.keras.layers.Dropout(rate=dropout)(outputs)
  outputs = tf.keras.layers.LayerNormalization(epsilon=1e-6)(attention + outputs)

  return tf.keras.Model(inputs=[inputs, padding_mask], outputs=outputs, name=name)

### Encoder
1. Input Embedding
2. Positional Encoding
3. `num_layers` encoder layers

Embedding + positional encoding : input

going encoder layers.

output going decoder

In [87]:
def encoder(vocab_size,
            num_layers,
            units,
            d_model,
            num_heads,
            dropout,
            name="encoder"):

  inputs = tf.keras.Input(shape=(None,), name="inputs")
  padding_mask = tf.keras.Input(shape=(1, 1, None), name="padding_mask")

  embeddings = tf.keras.layers.Embedding(vocab_size, d_model)(inputs)
  embeddings *= tf.math.sqrt(tf.cast(d_model, tf.float32))
  embeddings = PositionalEncoding(vocab_size, d_model)(embeddings)#??왜 vocab_size가 들어가지?

  outputs = tf.keras.layers.Dropout(rate=dropout)(embeddings)

  for i in range(num_layers):
    outputs = encoder_layer(
        units = units,
        d_model = d_model,
        num_heads = num_heads,
        dropout = dropout,
        name = "encoder_layer_{}".format(i),
    )([outputs, padding_mask])
  
  return tf.keras.Model(
      inputs = [inputs, padding_mask], outputs = outputs, name=name)

### Decoder Layer
1. Masked multi-head attention (with look ahead mask and padding mask)
2. Multi-head attention (with padding mask). `value` and `key` is from encoder output. `query` is from Multi-head attention layer output
3. 2 dense layers followed by dropout

also has residual connection followd by a layer normalization.

In [88]:
def decoder_layer(units, d_model, num_heads, dropout, name="decoder_layer"):
  inputs = tf.keras.Input(shape=(None, d_model), name="inputs")
  enc_outputs = tf.keras.Input(shape=(None, d_model), name="encoder_outputs")
  look_ahead_mask = tf.keras.Input(shape=(1,None,None), name="look_ahead_mask")
  padding_mask = tf.keras.Input(shape=(1,1,None), name="padding_mask")

  attention1 = MultiHeadAttention(
      d_model, num_heads, name="attention_1")(inputs={
          'query' : inputs,
          'key' : inputs,
          'value' : inputs,
          'mask' : look_ahead_mask
      })
  attention1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)(attention1 + inputs)

  attention2 = MultiHeadAttention(
      d_model, num_heads, name="attention_2")(inputs={
          'query' : attention1,
          'key' : enc_outputs,
          'value' : enc_outputs,
          'mask' : padding_mask
      })
  attention2 = tf.keras.layers.Dropout(rate=dropout)(attention2)
  attention2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)(attention2 + attention1)

  outputs = tf.keras.layers.Dense(units=units, activation='relu')(attention2)
  outputs = tf.keras.layers.Dense(units=d_model)(outputs)
  outputs = tf.keras.layers.Dropout(rate=dropout)(outputs)
  outputs = tf.keras.layers.LayerNormalization(epsilon=1e-6)(outputs + attention2)

  return tf.keras.Model(
      inputs = [inputs, enc_outputs, look_ahead_mask, padding_mask],
      outputs = outputs,
      name = name)

### Decoder
1. output Embedding
2. Positional Encoding
3. `num_layers` decoder layers

Embedding + positional encoding : input (target)

going decoder layers.

output going final linear layer

In [89]:
def decoder(vocab_size,
            num_layers,
            units,
            d_model,
            num_heads,
            dropout,
            name="decoder"):
  inputs = tf.keras.Input(shape=(None,), name="inputs")
  enc_outputs = tf.keras.Input(shape=(None, d_model), name="encoder_outputs")
  look_ahead_mask = tf.keras.Input(shape=(1,None,None), name="look_ahead_mask")
  padding_mask = tf.keras.Input(shape=(1,1,None), name='padding_mask')

  embeddings = tf.keras.layers.Embedding(vocab_size, d_model)(inputs)
  embeddings *= tf.math.sqrt(tf.cast(d_model, tf.float32))
  embeddings = PositionalEncoding(vocab_size, d_model)(embeddings)

  outputs = tf.keras.layers.Dropout(rate=dropout)(embeddings)

  for i in range(num_layers):
    outputs = decoder_layer(
        units = units,
        d_model = d_model,
        num_heads = num_heads,
        dropout = dropout,
        name = "decoder_layer_{}".format(i),
    )(inputs = [outputs, enc_outputs, look_ahead_mask, padding_mask])
  
  return tf.keras.Model(
      inputs = [inputs, enc_outputs, look_ahead_mask, padding_mask],
      outputs = outputs,
      name = name)

### Transformer
1. encoder
2. decoder
3. final linear layer

In [90]:
def transformer(vocab_size,
                num_layers,
                units,
                d_model,
                num_heads,
                dropout,
                name="transformer"):
  inputs = tf.keras.Input(shape=(None,), name="inputs")
  dec_inputs = tf.keras.Input(shape=(None,), name="dec_inputs")

  enc_padding_mask = tf.keras.layers.Lambda(
      create_padding_mask, output_shape=(1,1,None),
      name="enc_padding_mask")(inputs)
  
  #mask future tokens for decoder inputs at 1st attention block
  look_ahead_mask = tf.keras.layers.Lambda(
      create_look_ahead_mask, output_shape=(1,None,None),
      name="look_ahead_mask")(dec_inputs)
  
  #mask encoder outputs for the 2nd attention block
  dec_padding_mask = tf.keras.layers.Lambda(
      create_padding_mask, output_shape=(1,1,None),
      name="dec_padding_mask")(inputs)
  
  enc_outputs = encoder(
      vocab_size=vocab_size,
      num_layers=num_layers,
      units=units,
      d_model=d_model,
      num_heads=num_heads,
      dropout=dropout,
  )(inputs=[inputs, enc_padding_mask])

  dec_outputs = decoder(
      vocab_size=vocab_size,
      num_layers=num_layers,
      units=units,
      d_model=d_model,
      num_heads=num_heads,
      dropout=dropout,
  )(inputs=[dec_inputs, enc_outputs, look_ahead_mask, dec_padding_mask])

  outputs = tf.keras.layers.Dense(units=vocab_size, name="outputs")(dec_outputs)

  return tf.keras.Model(inputs=[inputs, dec_inputs], outputs=outputs, name=name)

## Transformer 생성
 - custom loss
 - custom learning rate

In [91]:
tf.keras.backend.clear_session()

model = transformer(
    vocab_size=VOCAB_SIZE,
    num_layers=NUM_LAYERS,
    units=UNITS,
    d_model=D_MODEL,
    num_heads=NUM_HEADS,
    dropout=DROPOUT)

### Loss function
since target sequences are padded, deal this.

In [92]:
def loss_function(y_true, y_pred):
  y_true = tf.reshape(y_true, shape=(-1, max_sequences))

  loss = tf.keras.losses.SparseCategoricalCrossentropy(
      from_logits=True, reduction='none')(y_true, y_pred)
  
  mask = tf.cast(tf.not_equal(y_true,0), tf.float32)
  loss = tf.multiply(loss, mask)

  return tf.reduce_mean(loss)

### Custom learning rate
use Adam optimizer with custom learning rate
$$\Large{lrate = d_{model}^{-0.5} * min(step{\_}num^{-0.5}, step{\_}num * warmup{\_}steps^{-1.5})}$$

In [93]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):

  def __init__(self, d_model, warmup_steps=4000):
    super(CustomSchedule, self).__init__()

    self.d_model = d_model
    self.d_model = tf.cast(self.d_model, tf.float32)

    self.warmup_steps = warmup_steps

  def get_config(self):
    config = {
        'd_model':self.d_model,
        'warmup_steps':self.warmup_steps
    }
    return config

  def __call__(self, step):
    arg1 = tf.math.rsqrt(step)
    arg2 = step * (self.warmup_steps**-1.5)

    return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

### Compile Model

In [94]:
learning_rate = CustomSchedule(D_MODEL)

optimizer = tf.keras.optimizers.Adam(
    learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

def accuracy(y_true, y_pred):
  # ensure labels have shape (batch_size, MAX_LENGTH - 1)
  y_true = tf.reshape(y_true, shape=(-1, max_sequences))
  return tf.keras.metrics.sparse_categorical_accuracy(y_true, y_pred)

model.compile(optimizer=optimizer, loss=loss_function, metrics=[accuracy])

## Train

In [95]:
model.fit(dataset, epochs=EPOCHS)

Epoch 1/50
115/115 [==============================] - 7s 65ms/step - loss: 1.9784 - accuracy: 0.0210
Epoch 2/50
115/115 [==============================] - 7s 65ms/step - loss: 1.7377 - accuracy: 0.0333
Epoch 3/50
115/115 [==============================] - 8s 69ms/step - loss: 1.4778 - accuracy: 0.0403
Epoch 4/50
115/115 [==============================] - 7s 65ms/step - loss: 1.3225 - accuracy: 0.0466
Epoch 5/50
115/115 [==============================] - 7s 65ms/step - loss: 1.2267 - accuracy: 0.0569
Epoch 6/50
115/115 [==============================] - 7s 65ms/step - loss: 1.1381 - accuracy: 0.0671
Epoch 7/50
115/115 [==============================] - 7s 65ms/step - loss: 1.0575 - accuracy: 0.0748
Epoch 8/50
115/115 [==============================] - 7s 65ms/step - loss: 0.9864 - accuracy: 0.0814
Epoch 9/50
115/115 [==============================] - 7s 65ms/step - loss: 0.9244 - accuracy: 0.0866
Epoch 10/50
115/115 [==============================] - 7s 65ms/step - loss: 0.8650 - accura

## Category Classification

In [26]:
# 분류모델에 들어갈 x 만들기
question_index_for_cl = []
answer_index_for_cl = []
question_index_for_cl = convert_text_to_index(question, word_to_index,0) #질문 문장에 정수인코딩,패딩
answer_index_for_cl = convert_text_to_index(answer, word_to_index,0)

question_index_for_cl = question_index_for_cl.tolist()
answer_index_for_cl = answer_index_for_cl.tolist()

question_index_for_cl.extend(answer_index_for_cl) # 이렇게 합쳐줌으로써 question_index_for_cl 이 결국 분류모델의 x로 들어가게 될 예정
print(question_index_for_cl)

[[3173, 4944, 3793, 55, 6124, 2879, 6011, 2048, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [4079, 3628, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [3197, 3983, 5343, 4604, 6275, 5619, 1660, 55, 4777, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [4208, 4828, 3345, 4777, 2811, 2048, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1079, 3379, 2048, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [5420, 4777, 3480, 5394, 5257, 2048, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [5394, 4944, 1120, 6092, 2048, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [3565, 2052, 4777, 5761, 4777, 3480, 3993, 4279, 2048, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [4828, 1228, 6228, 3287, 3379, 2048, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [2117, 1539, 

In [27]:
# 분류모델에 들어갈 y만들기
category_y = []
main_y = []
for i in faqs:
  category_y.append(i[3])
  main_y.append(i[4])

category_y.extend(category_y) # 자기것을 그대로 뒤에 갓다붙임. 그래도 되는게, x의 0번 인덱스의 문장(고객의 질문)과 7301번 인덱스의 문장(점원의 답변)이 카테고리,의도가 같음.
main_y.extend(main_y)

In [28]:
np.asarray(question_index_for_cl).shape

(14602, 30)

In [29]:
np.asarray(main_y).shape

(14602,)

In [30]:
np.asarray(question_index_for_cl).shape

(14602, 30)

In [31]:
np.asarray(category_y).shape

(14602,)

In [32]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding, Masking, Bidirectional

#의도 분류모델
mmodel = Sequential()
mmodel.add(Embedding(VOCAB_SIZE, 100))
mmodel.add(Masking(mask_value=0.0))
mmodel.add(Bidirectional(LSTM(128)))
mmodel.add(Dense(387, activation='softmax'))

#카테고리 분류모델
cmodel = Sequential()
cmodel.add(Embedding(VOCAB_SIZE, 100))
cmodel.add(Masking(mask_value=0.0))
cmodel.add(Bidirectional(LSTM(128)))
cmodel.add(Dense(4, activation='softmax'))

In [ ]:
mmodel.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['acc'])
history_m = mmodel.fit(np.asarray(question_index_for_cl), np.asarray(main_y), validation_split=0.1, batch_size=32, epochs=30)

In [85]:
mmodel.save('main_lstm_cl.h5')

In [ ]:
cmodel.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['acc'])
history_c = cmodel.fit(np.asarray(question_index_for_cl), np.asarray(category_y), validation_split=0.1, batch_size=32, epochs=30) # 128 64 32 실험

In [89]:
cmodel.save('category_lstm_cl.h5')

In [91]:
index_to_category

{0: '액세서리', 1: '신발', 2: '의류', 3: '가방'}

In [75]:
load_m_model = models.load_model('/content/drive/My Drive/main_lstm_cl_test.h5')
load_c_model = models.load_model('/content/drive/My Drive/category_lstm_cl_test.h5')

In [76]:
def find_category(stc,model,index_to_category, word_to_index):
    spaced_stc = pos_tag([stc])
    # print(spaced_stc)
    encoded_stc = convert_text_to_index(spaced_stc, word_to_index,0).reshape(1,30)
    # print(encoded_stc)
    score = model.predict(encoded_stc)
    # print(score.shape)
    return (index_to_category[score.argmax()], score[0, score.argmax()])

In [77]:
print(find_category("혹시 선호하는 색상 있으신가요?", load_m_model, index_to_main,word_to_index))
print(find_category("신발 얼만가요?", load_c_model, index_to_category,word_to_index))

('맞춤문의요청', 0.2650233)
('의류', 0.25530064)


## Predict

In [96]:
# 인덱스를 문장으로 변환
def convert_index_to_text(indexs, vocabulary): 
    
    sentence = ''
    
    # 모든 문장에 대해서 반복
    for index in indexs:
        if index == END_INDEX:
            # 종료 인덱스면 중지
            break;
        if vocabulary.get(index) is not None:
            # 사전에 있는 인덱스면 해당 단어를 추가
            sentence += vocabulary[index]
        else:
            # 사전에 없는 인덱스면 OOV 단어를 추가
            sentence.extend([vocabulary[OOV_INDEX]])
            
        # 빈칸 추가
        sentence += ' '

    return sentence

In [97]:
# 예측을 위한 입력 생성
def make_predict_input(sentence):

    sentences = []
    sentences.append(sentence)
    sentences = pos_tag(sentences)
    input_seq = convert_text_to_index(sentences, word_to_index, ENCODER_INPUT)
    
    return input_seq

In [98]:
def evaluate(input_seq):

  input_seq = input_seq.squeeze()
  sentence = tf.expand_dims(input_seq, axis=0)
  output = tf.expand_dims([1], 0)

  for i in range(max_sequences):
    predictions = model(inputs=[sentence, output], training=False)

    # select the last word from the seq_len dimension
    predictions = predictions[:, -1:, :]
    predicted_id = tf.cast(tf.argmax(predictions, axis=-1), tf.int32)

    # return the result if the predicted_id is equal to the end token
    if tf.equal(predicted_id, 2):
      break

    # concatenated the predicted_id to the output which is given to the decoder
    # as its input.
    output = tf.concat([output, predicted_id], axis=-1)

  return tf.squeeze(output, axis=0)

In [99]:
test_sentece = "은침 있나요?"
print(test_sentece)
print(find_category(test_sentece,cmodel,index_to_category,word_to_index))
print(find_category(test_sentece, mmodel,index_to_main,word_to_index))
print()

tmp_test_stc = convert_index_to_text(evaluate(make_predict_input(test_sentece)).numpy()[1:],index_to_word)
print(tmp_test_stc)
print(find_category(tmp_test_stc,cmodel,index_to_category,word_to_index))
print(find_category(tmp_test_stc, mmodel,index_to_main,word_to_index))

은침 있나요?
('가방', 0.25901893)
('소재를제시한제품문의', 0.78608567)

이 제품 은 은침 은 아니에요 
('가방', 0.25875258)
('소재를제시한제품문의', 0.97787285)


##doc2vec

In [100]:
import os
from gensim.models import doc2vec
from gensim.models.doc2vec import TaggedDocument
import pandas as pd
import jpype
from konlpy.tag import Kkma

In [101]:
import warnings
warnings.filterwarnings(action='ignore') 

In [102]:
kkma = Kkma()
filter_kkma = ['NNG', 'NNP','OL','VA','VV','VXV']

def tokenizer_kkma(doc):
    # 꼬꼬마 형태소 분석기가 자바 기반이어서 파이썬에서 자바함수들을 실행할 수 있는 명령어 (jpype) 를 써줘야한다.
    jpype.attachThreadToJVM()       
    token_doc = ["/".join(word) for word in kkma.pos(doc)]
    return token_doc

def tokenize_kkma_noun_verb(doc):
    jpype.attachThreadToJVM()
    token_doc = ["/".join(word) for word in kkma.pos(doc) if word[1] in filter_kkma]
    return token_doc

In [103]:
token_faqs = [(tokenizer_kkma(row[1]), row[0]) for row in faqs]
tagged_faqs = [TaggedDocument(d,[c]) for d,c in token_faqs]

In [104]:
# 모델 만들기
# cpu 몇 개 쓸 건지
import multiprocessing
# 내 컴에 있는 cpu 갯수 cores 에 저장
cores = multiprocessing.cpu_count()
# vector_size : 임베딩할 벡터 차원
# negaive : negative sampling
d2v_faqs = doc2vec.Doc2Vec(
    vector_size = 100,
    alpha = 0.025,
    min_alpha = 0.025,
    hs = 1,
    negative = 0,
    dm = 0,
    dbow_words = 1,
    min_count = 1,
    workers = cores,
    seed = 0
)

# 단어 사전 만들기
d2v_faqs.build_vocab(tagged_faqs)
for epoch in range(4):
  # 모델 학습
  print(epoch)
  d2v_faqs.train(tagged_faqs,
                 total_examples = d2v_faqs.corpus_count,
                 epochs = d2v_faqs.epochs)
  d2v_faqs.alpha -=0.0025
  d2v_faqs.min_alpha = d2v_faqs.min_alpha

0
1
2
3


# spell check and spacing

In [105]:
def grammar_checker(sentence):

  spacing_sentence = spacing(sentence.replace(' ',''))
  spelled_sentence = spell_checker.check(spacing_sentence)
  checked_sentence = spelled_sentence.checked

  return checked_sentence

##합치기

In [158]:
def give_cate(input_question): #카테고리추출하는 함수
  big_cate=find_category(input_question,cmodel,index_to_category,word_to_index)
  samll_cate=find_category(input_question,mmodel,index_to_main,word_to_index)
  return big_cate,samll_cate #리턴값으로 큰카테고리(이름,유사도),작은 카테고리(이름,유사도)

def give_answer(input_question): #질문입력시 transfomer로 답변함
  return convert_index_to_text(evaluate(make_predict_input(input_question)).numpy()[1:],index_to_word)

def doc2_answer(input_question): #질문 입력시 doc2으로 답변함
  token_test = tokenizer_kkma(input_question)
  predict_vector = d2v_faqs.infer_vector(token_test)
  result = d2v_faqs.docvecs.most_similar([predict_vector],topn=1)
  return faqs[int(result[0][0])-1][2]

def score_calcul(left_cate,right_cate, num):#카테고리를 두개를 입력하면 유사도를 계산함
  result = 0
  if left_cate[0]==right_cate[0]:
      if round(abs(left_cate[1]-right_cate[1]),3) == 0:
          result+=1000
      else:
        #   print(abs(left_cate[1]-right_cate[1]))
          result += (1/round(abs(left_cate[1]-right_cate[1]),3)) # 최대 999 이상 나올 수 없다.

  if num == 1 and result > 0:
      result+=1000
  return result

In [162]:
def result_final(stc):
    q_big_cate,q_small_cate=give_cate(stc)
    print(f"질문 카테고리 : {q_big_cate}, 질문 의도 : {q_small_cate}")
    
    dm=doc2_answer(stc)
    dm_big_cate,dm_small_cate=give_cate(dm)
    print(f"독투벡 답변 : {dm}\n독투벡 카테고리 : {dm_big_cate}, 독투벡 의도 : {dm_small_cate}")
    
    cm=give_answer(input_question)
    cm_big_cate,cm_small_cate=give_cate(cm)
    print(f"트랜 답변 : {cm}\n트랜 카테고리 : {cm_big_cate}, 트랜 의도 : {cm_small_cate}")

    doc2_score=0
    tran_score=0

    doc2_score += score_calcul(q_big_cate,dm_big_cate,0)
    doc2_score += score_calcul(q_small_cate,dm_small_cate,1)
    tran_score += score_calcul(q_big_cate,cm_big_cate,0)
    tran_score += score_calcul(q_small_cate,cm_small_cate,1)


    print(f"doc2 score : {doc2_score}, tran score : {tran_score}")

    if doc2_score == 0 and trans_score == 0:
        print("잘 모르겠습니다.")
        return

    if doc2_score>=tran_score:
        print(f"***doc2: {grammar_checker(dm)}")
    else:
        print(f"***tran: {grammar_checker(cm)}")

    return

result_final("화장실 어디에 있나요?")


질문 카테고리 : ('가방', 0.25625175), 질문 의도 : ('제품위치문의', 0.4203108)
독투벡 답변 : 여기 밖에 있어요
독투벡 카테고리 : ('가방', 0.25943258), 독투벡 의도 : ('화장실위치문의', 0.37811407)
트랜 답변 : 본인 이 입으시는 건가 요 ? 
트랜 카테고리 : ('가방', 0.25702974), 트랜 의도 : ('종류별의류제품문의요청', 0.9953726)
doc2 score : 333.33333043588533, tran score : 999.999952502551
***tran: 본인이 입으시는 건가요?
